In [1]:
%load_ext autoreload
%autoreload 2

# Verify conditional circuit generation

We have previously trained a neural network known as a conditional variational autoencoder on genetic circuits. The conditioning variable was made to be one of the characteristics of the genetic circuit's dynamics, for example the sensitivity of the circuit to a signal. While the distributions for each class of conditional variables differ in the AI-generated circuits, the verification of their ground-truth label is still necessary to assess the validity of the conditionality.

## Imports

In [2]:
from synbio_morpher.srv.io.manage.script_manager import script_preamble
from synbio_morpher.srv.parameter_prediction.simulator import RawSimulationHandling, make_piecewise_stepcontrol
from synbio_morpher.utils.results.analytics.timeseries import generate_analytics
from synbio_morpher.utils.common.setup import prepare_config, expand_config, expand_model_config
from synbio_morpher.utils.data.data_format_tools.common import load_json_as_dict
from synbio_morpher.utils.results.analytics.naming import get_true_interaction_cols
from synbio_morpher.utils.data.data_format_tools.common import write_json
from synbio_morpher.utils.misc.numerical import symmetrical_matrix_length, make_symmetrical_matrix_from_sequence
from synbio_morpher.utils.misc.string_handling import prettify_keys_for_label
from synbio_morpher.utils.misc.type_handling import flatten_listlike, get_unique
from synbio_morpher.utils.modelling.deterministic import bioreaction_sim_dfx_expanded
from bioreaction.simulation.simfuncs.basic_de import bioreaction_sim, bioreaction_sim_expanded
from bioreaction.model.data_tools import construct_model_fromnames
from bioreaction.model.data_containers import BasicModel, QuantifiedReactions
from bioreaction.simulation.manager import simulate_steady_states
from functools import partial

from sklearn.metrics import r2_score  
import os
import sys
import numpy as np
import haiku as hk
import jax
import diffrax as dfx

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
                
import wandb

from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

jax.config.update('jax_platform_name', 'cpu')


# if __package__ is None:

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

__package__ = os.path.basename(module_path)


jax.devices()

/home/wadh6511/Kode/env_evo/lib/python3.10/site-packages/jaxtyping/__init__.py:221: UserWarning: jaxtyping version >=0.2.23 should be used with Equinox version >=0.11.1
  warnings.warn(
I0000 00:00:1702849086.471888   31807 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
xla_bridge.py:backends():513: Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA INFO
xla_bridge.py:backends():513: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory INFO


[CpuDevice(id=0)]

In [3]:
from src.models.vae import CVAE, sample_z
from src.models.mlp import MLP
from src.models.shared import arrayise
from src.losses.losses import loss_wrapper, compute_accuracy_regression, mse_loss, accuracy_regression
from src.utils.data_preprocessing import drop_duplicates_keep_first_n
from src.utils.optimiser import make_optimiser
from src.utils.train import make_saves

## Load data

In [4]:
fn = '../data/processed/ensemble_mutation_effect_analysis/2023_07_17_105328/tabulated_mutation_info.csv'
fn_test_data = '../data/raw/ensemble_mutation_effect_analysis/2023_10_03_204819/tabulated_mutation_info.csv'
data = pd.read_csv(fn)
try:
    data.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass

## Load model

### Hyperparameters

In [5]:
# Architecture
HIDDEN_SIZE = 32
NUM_ENC_LAYERS = 3
NUM_DEC_LAYERS = 3


BATCH_SIZE = 128
N_BATCHES = 1200
TOTAL_DS = BATCH_SIZE * N_BATCHES
MAX_TOTAL_DS = TOTAL_DS
TRAIN_SPLIT = 0.8
SCALE_X = False
LEARNING_RATE = 5e-4
LEARNING_RATE_SCHED = 'cosine_decay'
# LEARNING_RATE_SCHED = 'constant'
WARMUP_EPOCHS = 20
L2_REG_ALPHA = 0.01
EPOCHS = 1000
PRINT_EVERY = EPOCHS // 100
SEED = 1

INPUT_SPECIES = 'RNA_1'
USE_CATEGORICAL = False
target_circ_func = 'sensitivity_wrt_species-6'
input_concat_diffs = False
input_concat_axis = 0

# Training
USE_DROPOUT = False
USE_L2_REG = False
USE_WARMUP = True

loss_fn = partial(
    loss_wrapper, loss_f=mse_loss, use_l2_reg=USE_L2_REG) 
compute_accuracy = compute_accuracy_regression

rng = jax.random.PRNGKey(SEED)

### Init data

In [6]:
filt = data['sample_name'] == INPUT_SPECIES

# Balance the dataset
df = drop_duplicates_keep_first_n(data[filt], get_true_interaction_cols(
    data, 'energies', remove_symmetrical=True), n=100)

TOTAL_DS = np.min([TOTAL_DS, MAX_TOTAL_DS, len(df)])
N_BATCHES = TOTAL_DS // BATCH_SIZE
TOTAL_DS = N_BATCHES * BATCH_SIZE

x_cols = [get_true_interaction_cols(data, 'energies', remove_symmetrical=True)]
if input_concat_diffs:
    k = 'energies'
    x_cols = x_cols + \
        [[f'{i}_diffs' for i in get_true_interaction_cols(
            data, k, remove_symmetrical=True)]]

x = [df[i].iloc[:TOTAL_DS].values[:, :, None] for i in x_cols]
x = np.concatenate(x, axis=input_concat_axis+1).squeeze()

if SCALE_X:
    xscaler = MinMaxScaler()
    x = xscaler.fit_transform(x)
    
cond = df[target_circ_func].iloc[:TOTAL_DS].to_numpy()

if USE_CATEGORICAL:
    from src.utils.math import convert_to_scientific_exponent

    vectorized_convert_to_scientific_exponent = np.vectorize(
        convert_to_scientific_exponent)
    numerical_resolution = 2
    cond_map = {k: numerical_resolution for k in np.arange(int(f'{cond[cond != 0].min():.0e}'.split(
        'e')[1])-1, np.max([int(f'{cond.max():.0e}'.split('e')[1])+1, 0 + 1]))}
    cond_map[-6] = 1
    cond_map[-5] = 1
    cond_map[-4] = 4
    cond_map[-3] = 2
    cond_map[-1] = 3
    cond = jax.tree_util.tree_map(partial(
        vectorized_convert_to_scientific_exponent, numerical_resolution=cond_map), cond)
    cond = np.interp(cond, sorted(np.unique(cond)), np.arange(
        len(sorted(np.unique(cond))))).astype(int)
else:
    zero_log_replacement = -10.0
    cond = np.where(cond != 0, np.log10(cond), zero_log_replacement)

cond = cond[:, None]
N_HEAD = x.shape[-1]

x, cond = shuffle(x, cond, random_state=SEED)

if x.shape[0] < TOTAL_DS:
    print(
        f'WARNING: The filtered data is not as large as the requested total dataset size: {x.shape[0]} vs. requested {TOTAL_DS}')

### Init model

In [7]:
enc_layers = [64] * NUM_ENC_LAYERS
dec_layers = [64] * NUM_DEC_LAYERS

def VAE_fn(enc_layers: list, dec_layers: list, call_kwargs: dict = {}):
    encoder = MLP(layer_sizes=enc_layers, n_head=dec_layers[0], use_categorical=False, name='encoder')
    decoder = MLP(layer_sizes=dec_layers, n_head=x.shape[-1], use_categorical=False, name='decoder')
    model = CVAE(encoder=encoder, decoder=decoder, embed_size=HIDDEN_SIZE)
    
    def init(x: np.ndarray, cond: np.ndarray, deterministic: bool):
        h = model.encoder(np.concatenate([x, cond], axis=-1))

        mu = model.h2mu(h)
        logvar = model.h2logvar(h)
        z = model.reparameterize(mu, logvar, hk.next_rng_key(), deterministic)
        z_cond = np.concatenate([z, cond], axis=-1)

        y = model.decoder(z_cond)
        return y
        
    return init, (encoder, decoder, model)

model_fn = partial(VAE_fn, enc_layers=enc_layers, dec_layers=dec_layers)
# model = hk.transform(model_fn)
model_t = hk.multi_transform(model_fn)
params = model_t.init(rng, x, cond, deterministic=False)
encoder, decoder, model = model_t.apply

/home/wadh6511/Kode/env_evo/lib/python3.10/site-packages/haiku/_src/initializers.py:126: UserWarning: Explicitly requested dtype float64  is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  unscaled = jax.random.truncated_normal(
/home/wadh6511/Kode/env_evo/lib/python3.10/site-packages/haiku/_src/base.py:682: UserWarning: Explicitly requested dtype float64 requested in zeros is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  param = init(shape, dtype)


In [8]:
fn_saves = os.path.join('weight_saves', '10_cvae', '2023_12_14__11_37_03_saves_test')
saves_loaded = load_json_as_dict(fn_saves)

p = saves_loaded[str(list(saves_loaded.keys())[-1])]['params']
p = arrayise(p)

### Sample circuits

In [9]:
n_to_sample = 10000

sampled_cond = np.interp(np.random.rand(n_to_sample, cond.shape[-1]), [0, 1], [cond.min(), cond.max()])
mu = np.random.normal(size=(n_to_sample, HIDDEN_SIZE)) 
logvar = np.random.normal(size=(n_to_sample, HIDDEN_SIZE)) 
z = sample_z(mu=mu, logvar=logvar, key=rng)
z = np.concatenate([z, sampled_cond], axis=-1)

fake_circuits = decoder(p, rng, z)
fake_circuits = np.where(fake_circuits > 0, 0, fake_circuits)
sampled_cond.shape

(10000, 1)

# Simulate 

## Construct circuits

### Config

In [10]:
run_intarna = True

config = {
    "system_type": "RNA",
    "include_prod_deg": False,
    "data": {},
    "experiment": {
        "purpose": "tests",
        "debug_mode": False
    },
    "interaction_simulator": {
        "name": "IntaRNA",
        "postprocess": True,
        "simulator_kwargs": {
            "outcsvcols": "id1, id2, E, E_norm, bpList, hybridDPfull, seedPu1, seedPu2, seedStart1, seedStart2, seedEnd1, seedEnd2",
            "threads": 12,
            "n": 1,
            "raw_stdout": True
        }
    },
    "molecular_params_factor": 1,
    "molecular_params": {
        "avg_mRNA_per_cell": 100,
        "cell_doubling_time": 1200,
        "creation_rate": 2.35,
        "starting_copynumbers": 200,
        "degradation_rate": 0.01175,
        "association_binding_rate": 1000000
    },
    "signal": {
        "inputs": ["RNA_0"],
        "outputs": ["RNA_1"],
        "function_name": "step_function",
        "function_kwargs": {
            "impulse_center": 1, 
            "impulse_halfwidth": 5, 
            "target": 2
        }
    },
    "simulation": {
        "dt0": 0.1,
        "t0": 0,
        "t1": 100,
        "tmax": 2000,
        "solver": "diffrax",
        "use_batch_mutations": True,
        "interaction_factor": 1,
        "batch_size": 20000,
        "max_circuits": 60000,
        "device": "gpu",
        "threshold_steady_states": 0.05, 
        "use_rate_scaling": True
    },
    "system_type": "RNA",
}


if run_intarna:
    data_writer = None
    config, data_writer = script_preamble(config, data_writer)
    config = prepare_config(expand_config(config=config))

### Circuits

In [11]:
def make_symmetrical_matrix_from_sequence_nojax(arr, side_length: int):
    """ For a flat 1D array, make a symmetrical 2D matrix filling
    in the upper triangle with the 1D array. Not jax-friendly. """
    n = np.zeros((side_length, side_length))
    n[np.triu_indices(side_length)] = arr
    return n

def update_species_simulated_rates(ordered_species: list, forward_interactions, reverse_interactions, model: BasicModel, qreactions: QuantifiedReactions):
    for i, r in enumerate(model.reactions):
        if len(r.input) == 2:
            model.reactions[i].forward_rate = forward_interactions[
                ordered_species.index(r.input[0]), ordered_species.index(r.input[1])]
            
            model.reactions[i].reverse_rate = reverse_interactions[
                ordered_species.index(r.input[0]), ordered_species.index(r.input[1])]
    qreactions.reactions = qreactions.init_reactions(
        model)
    return model, qreactions

Reshape the generated circuit energies into a 3x3 symmetrical matrix.

In [12]:
num_species = symmetrical_matrix_length(fake_circuits.shape[-1])
input_species = [f'RNA_{i}' for i in range(num_species)]
config.update(expand_model_config(config, {}, input_species))
fake_circuits = np.array(list(map(partial(make_symmetrical_matrix_from_sequence_nojax, side_length=num_species), fake_circuits)))

Initialise barebones circuit models

In [13]:
model_brn = construct_model_fromnames(sample_names=input_species, include_prod_deg=config['include_prod_deg'])
ordered_species = sorted(get_unique(flatten_listlike([r.input for r in model_brn.reactions if r.output])))
signal_species = ['RNA_0']
output_species = ['RNA_1']
qreactions = QuantifiedReactions()
qreactions.init_properties(model_brn, config['starting_concentration'])
for i in range(len(model_brn.reactions)):
    model_brn.reactions[i].forward_rate = 0
    model_brn.reactions[i].reverse_rate = 0
    if (not model_brn.reactions[i].input) and config['include_prod_deg']:
        model_brn.reactions[i].forward_rate = config['molecular_params'].get(
            'creation_rate')
    elif (not model_brn.reactions[i].output) and config['include_prod_deg']:
        model_brn.reactions[i].forward_rate = config['molecular_params'].get(
            'degradation_rate')
        

reactant_species = [
    r.species for r in qreactions.reactants]
quantities = [r.quantity for r in qreactions.reactants if r.species.name in input_species]
postproc = RawSimulationHandling(config['interaction_simulator']).get_postprocessing(initial=quantities)
eqconstants, (a_rates, d_rates) = postproc(fake_circuits[0])
model_brn, qreactions = update_species_simulated_rates(ordered_species, a_rates, d_rates, model_brn, qreactions)


## Run simulator

In [14]:
def mini_sim(B11, B12, B13, B22, B23, B33):
    """ Only works for circuits with size n = 3 """
    unbound_species = ['RNA_0', 'RNA_1', 'RNA_2']
    species = ['RNA_0-0', 'RNA_0-1', 'RNA_0-2', 'RNA_1-1',
               'RNA_1-2', 'RNA_2-2', 'RNA_0', 'RNA_1', 'RNA_2']
    signal_species = ['RNA_0']
    output_species = ['RNA_1']
    s_idxs = [species.index(s) for s in signal_species]
    output_idxs = [species.index(s) for s in output_species]
    signal_onehot = np.array([1 if s in [species.index(
        ss) for ss in signal_species] else 0 for s in np.arange(len(species))])

    signal_target = 2
    k = 0.00150958097
    N0 = 200

    # Amounts
    y00 = np.array([[0, 0, 0, 0, 0, 0, N0, N0, N0]])

    # Reactions
    inputs = np.array([
        [0, 0, 0, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 2, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 2],
    ])
    outputs = np.array([
        [1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0],
    ])

    # Rates
    reverse_rates = np.array([[B11, B12, B13, B22, B23, B33]])
    forward_rates = np.ones_like(reverse_rates) * k

    # Sim params
    t0 = 0
    t1 = 100
    # dt0 = scale_rates(forward_rates, reverse_rates, cushioning=4)
    dt0 = 0.0005555558569638981
    dt1_factor = 5
    dt1 = dt0 * dt1_factor
    max_steps = 16**4 * 10
    sim_func = jax.jit(partial(bioreaction_sim_dfx_expanded,
                               t0=t0, t1=t1, dt0=dt0,
                               signal=vanilla_return, signal_onehot=1,
                               forward_rates=forward_rates,
                               inputs=inputs,
                               outputs=outputs,
                               solver=dfx.Tsit5(),
                               saveat=dfx.SaveAt(
                                   ts=np.linspace(t0, t1, 500)),  # int(np.min([500, self.t1-self.t0]))))
                               max_steps=max_steps,
                               stepsize_controller=make_piecewise_stepcontrol(
                                   t0=t0, t1=t1, dt0=dt0, dt1=dt1)
                               ))

    s = analytics[f'sensitivity_wrt_species-{s_idxs[0]}']
    p = analytics[f'precision_wrt_species-{s_idxs[0]}']
    r = optimise_sp(
        s=s[:, None][tuple(output_idxs)], p=p[:, None][tuple(output_idxs)]
    )[0]

    return r, analytics, y, t

In [15]:
def compute_analytics(y, t, labels, signal_onehot):
    y = np.swapaxes(y, 0, 1)

    analytics_func = partial(
        generate_analytics, time=t, labels=labels,
        signal_onehot=signal_onehot, signal_time=t[1],
        ref_circuit_data=None)
    return analytics_func(data=y, time=t, labels=labels)


def sim(y00, forward_rates, reverse_rates,
        qreactions,
        signal_onehot, signal_target,
        t0, t1, dt0, dt1,
        save_steps, max_steps):
    sim_func = jax.jit(jax.vmap(partial(bioreaction_sim_dfx_expanded,
                                t0=t0, t1=t1, dt0=dt0,
                                signal=None, signal_onehot=None,
                                inputs=qreactions.reactions.inputs,
                                outputs=qreactions.reactions.outputs,
                                forward_rates=forward_rates,
                                solver=dfx.Tsit5(),
                                saveat=dfx.SaveAt(
                                    ts=np.linspace(t0, t1, save_steps)),
                                max_steps=max_steps,
                                stepsize_controller=make_piecewise_stepcontrol(
                                    t0=t0, t1=t1, dt0=dt0, dt1=dt1)
                                )))

    y0, t = simulate_steady_states(y0=y00, total_time=t1-t0, sim_func=sim_func, t0=t0,
                                   t1=t1, threshold=0.05, reverse_rates=reverse_rates, disable_logging=True)
    y0 = np.array(y0.squeeze()[-1, :]).reshape(y00.shape)

    # Signal
    y0s = y0 * ((signal_onehot == 0) * 1) + y00 * signal_target * signal_onehot
    y, t = simulate_steady_states(y0s, total_time=t1-t0, sim_func=sim_func, t0=t0,
                                  t1=t1, threshold=0.05, reverse_rates=reverse_rates, disable_logging=True)
    y = np.concatenate([y0, y.squeeze()[:-1, :]], axis=0)

    analytics = compute_analytics(y, t, labels=np.arange(
        y.shape[-1]), signal_onehot=signal_onehot)

    return analytics, y, t, y0s

In [22]:
signal_onehot = np.where([r.species.name in input_species for r in qreactions.reactants], 1, 0)
eqconstants, (forward_rates, reverse_rates) = postproc(fake_circuits)
forward_rates, reverse_rates = np.array(list(map(lambda i: i[np.triu_indices(n=forward_rates.shape[-1])], forward_rates))), np.array(list(map(lambda i: i[np.triu_indices(n=reverse_rates.shape[-1])], reverse_rates)))
signal_target = 2
y00 = np.repeat(np.array([r.quantity for r in qreactions.reactants])[None, None, :], repeats=len(fake_circuits), axis=0)
t0, t1, dt0, dt1 = 0, 100, 0.01, 0.01
save_steps, max_steps = 10, 16**5

analytics, y, t, y0s = sim(y00, forward_rates[0], reverse_rates,
    qreactions,
    signal_onehot, signal_target,
    t0, t1, dt0, dt1,
    save_steps, max_steps)